Desafio 1 <br>
Aluno: Júlio César Werner Scholz <br>
RA: 2023890 <br>

In [81]:

import pandas as pd
import json
import numpy as np
import folium
from folium.plugins import HeatMap

#import sys
#!{sys.executable} -m pip install pandas-profiling
import pandas_profiling as pp

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [82]:
reviews = pd.read_csv("reviewsTrainToronto.csv")
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447110 entries, 0 to 447109
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      447110 non-null  object
 1   business_id  447110 non-null  object
 2   useful       447110 non-null  int64 
 3   funny        447110 non-null  int64 
 4   cool         447110 non-null  int64 
 5   text         447110 non-null  object
 6   date         447110 non-null  object
dtypes: int64(3), object(4)
memory usage: 23.9+ MB


In [83]:
business = pd.read_csv("X_TrainToronto.csv")
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16070 entries, 0 to 16069
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   16070 non-null  object 
 1   name          16070 non-null  object 
 2   address       15806 non-null  object 
 3   postal_code   15967 non-null  object 
 4   latitude      16070 non-null  float64
 5   longitude     16070 non-null  float64
 6   review_count  16070 non-null  int64  
 7   is_open       16070 non-null  int64  
 8   attributes    13709 non-null  object 
 9   categories    16045 non-null  object 
 10  hours         11818 non-null  object 
 11  loc           16070 non-null  object 
 12  destaque      16070 non-null  int64  
dtypes: float64(2), int64(3), object(8)
memory usage: 1.6+ MB


In [84]:
""" # Aqui será gerado um novo dataframe contendo somente os atributos como colunas
#os jsons na coluna attributes precisam de um tratamento pois existem algumas inconsistencias

business['attributes'] = business['attributes'].fillna('{}')
business['attributes'] = business['attributes'].str.replace('\"','')
business['attributes'] = business['attributes'].str.replace('\'','\"')
business['attributes'] = business['attributes'].str.replace(' False','"False"')
business['attributes'] = business['attributes'].str.replace(' True','"True"')
business['attributes'] = business['attributes'].str.replace('u"','\"')

business.drop('address',axis=1, inplace=True)
business.drop('postal_code',axis=1, inplace=True)
		
df_json = pd.json_normalize(business.attributes.apply(json.loads))
#df_json = pd.concat([df_json,business['destaque']],axis=1)

business.drop('attributes',axis=1, inplace=True) """


' # Aqui será gerado um novo dataframe contendo somente os atributos como colunas\n#os jsons na coluna attributes precisam de um tratamento pois existem algumas inconsistencias\n\nbusiness[\'attributes\'] = business[\'attributes\'].fillna(\'{}\')\nbusiness[\'attributes\'] = business[\'attributes\'].str.replace(\'"\',\'\')\nbusiness[\'attributes\'] = business[\'attributes\'].str.replace(\'\'\',\'"\')\nbusiness[\'attributes\'] = business[\'attributes\'].str.replace(\' False\',\'"False"\')\nbusiness[\'attributes\'] = business[\'attributes\'].str.replace(\' True\',\'"True"\')\nbusiness[\'attributes\'] = business[\'attributes\'].str.replace(\'u"\',\'"\')\n\nbusiness.drop(\'address\',axis=1, inplace=True)\nbusiness.drop(\'postal_code\',axis=1, inplace=True)\n\t\t\ndf_json = pd.json_normalize(business.attributes.apply(json.loads))\n#df_json = pd.concat([df_json,business[\'destaque\']],axis=1)\n\nbusiness.drop(\'attributes\',axis=1, inplace=True) '

In [85]:
business['attributes'] = business['attributes'].fillna('{}')
business['attributes'] = business['attributes'].str.replace('\"','')
business['attributes'] = business['attributes'].str.replace('\'','\"')
business['attributes'] = business['attributes'].str.replace(' False','"False"')
business['attributes'] = business['attributes'].str.replace(' True','"True"')
business['attributes'] = business['attributes'].str.replace('u"','\"')
		
df_json = pd.json_normalize(business.attributes.apply(json.loads))


In [86]:

#df_json = pd.concat([df_json,business['destaque']],axis=1)
limitPer = len(df_json) * .50
aux = df_json.dropna(thresh=limitPer, axis=1)
pp.ProfileReport(aux)

#OBS: Professor tive um problema que nao encontrei a causa para gerar a visualização apenas com os estabelcimentos com destaque 1,
#   com isso estou disponibilizando o profiling de todos os estabelecimentos
#   os dados mencionados no markdown abaixo são relativos ao profiling que buguou e nao consegui mais gerar


Render HTML: 100%|██████████| 1/1 [00:00<00:00,  3.50it/s]


Coluna Atributos, apenas dos estabelecimentos com destaque 1. <br>
A coluna atributos foi tratada para que cada atributo individual se tornasse uma coluna no dataframe tendo como as linhas o valor especifico para cada estabelecimento<br>

Foram contados 87* diferentes atributos. A grande maioria deles não estando presentes nos estabelecimentos. Apenas 7 atributos possuem menos de 50% de valores faltantes, são eles:<br>

BikeParking <br>
RestaurantsPriceRange2 <br>
BusinessParking.garage	<br>
BusinessParking.street	<br>
BusinessParking.validated <br>	
BusinessParking.lot	<br>
BusinessParking.valet <br>

Nota-se que atributos relacionados a estacionamento de automoveis e bicicletas são os mais comuns. E o mais comum sendo a faixa de preço com uma presença de 70,5%. <br>

A faixa de preço varia de 4 a 1, sendo a mais comum é a de valor 2 com uma frequência de 41,7% ou seja 2686 estabelecimentos. <br>
A faixa de preço mais incomum é a 4 com apenas 97 estabelcimentos (1,5%)<br>



In [87]:
#a partir dos dados de reviews uma nova coluna sera gerada, que é a coluna total com a soma dos três tipos de reviews.

reviews_sum = reviews.groupby('business_id').sum()
reviews_sum['total'] = reviews_sum[['useful','funny','cool']].sum(1)

reviews_sum = reviews_sum.sort_values(by=['total'], ascending=False)
#print(reviews_sum['total'].median())

print(reviews_sum.head(4)) 
reviews_sum.describe()


                        useful  funny  cool  total
business_id                                       
RtUvSWO_UZ8V3Wpj0n077w    1558    839  1139   3536
-9dmhyBvepc08KPEHlEM0w     938    531   701   2170
aLcFhMe6DDJ430zelCpd2A    1138    424   525   2087
AUv2UoAAi7VASJ6VK5lOlw     917    459   710   2086


,useful,funny,cool,total
count,16070.000000,16070.000000,16070.000000,16070.000000
mean,38.139390,12.314126,15.298258,65.751773
std,75.966258,29.186696,37.473749,137.127336
min,0.000000,0.000000,0.000000,0.000000
25%,5.000000,1.000000,1.000000,8.000000
50%,14.000000,4.000000,4.000000,22.000000
75%,39.000000,12.000000,14.000000,65.000000
max,1558.000000,839.000000,1139.000000,3536.000000



O alto desvio padrão de todas as categorias de reviews indica que as reviews estão espalhadas por uma ampla gama de valores e é o que acontece, temos estabelicimentos desde 0 reviews até 3536 reviews.<br>
50% dos estabelecimentos possuem no maximo 22 reviews.<br>
484 estabelicimentos não possuem nenhuma review, e a categoria de review mais comum é a "useful"

In [88]:
pp.ProfileReport(reviews_sum)

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


In [89]:
business = business.set_index("business_id")
df = business.join(reviews_sum, how='outer')

Analisando o mapa de calor, com a localização dos estabelcimentos com destaque.<br>
Eles estão bem espelhados por Toronto, com uma maior concentração na parte Sul e central da cidade.<br>
A rua Queen Street West sendo a mais populosa em termos de quantidade de estabelcimentos com destaque. <br>
Para fazer a predição se um estabeliciemtno é bem avaliado, seria interessante dividir a cidade em grids de mesmo tamanho e assim considerar se um estabelciemento esta dentro de um grid com outros locais bem avaliados a chance de ter destaque pode aumentar.

In [90]:
mapaToronto = folium.Map(location=[43.667996, -79.408770])
pontos  = [[row['latitude'],row['longitude']] for index, row in df.iterrows()  if row['destaque'] == 1 ]

HeatMap(pontos).add_to(mapaToronto)
mapaToronto

In [91]:
#Split da coluna categorias em multiplas colunas, uma para cada categoria contendo um valor booleano, true se o estabelcimento tem a categoria, falso caso contrario
df_categ = business['categories'].str.get_dummies(sep=', ').astype(bool)
destaques = business['destaque'].astype(bool)

#Criaçã da categoria sem destaque, é o inverso da coluna destaque
df_categ = pd.concat([df_categ,business['destaque']], axis=1)
df_categ['sem destaque'] = np.invert(destaques) 

#Coluna do intervalo de preço
col_names = df_json['RestaurantsPriceRange2'].dropna().unique().tolist()
df_categ[col_names] = pd.get_dummies(df_json['RestaurantsPriceRange2'])

df_categ.dropna(inplace=True, axis=1)



In [92]:
#Escolhi um suporte minimo pois o dataset é bem diverso, assim mais itens frequentes foram gerados.
frequent_itemsets = apriori(df_categ, min_support=0.005, use_colnames=True)


Das regras de associação que foram geradas, as que mais possuem confiação e a consequência é um estabelcimento de destaque. <br><br>

A categoria health & Medical e relacionadas como Acupuncture, Massage Therapy)<br>
(Shopping, Arts & Entertainment)<br>
(Shopping, Event Planning & Services)	

In [93]:
#Regras com o valor de confiança mínimo escolhido
dfRegras= association_rules(frequent_itemsets, metric='confidence', min_threshold=0.65)
dfRegras = dfRegras.sort_values(by='confidence',ascending=False)
dfRegras[(dfRegras['consequents'] == {'destaque'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
235,"(Acupuncture, Health & Medical)",(destaque),0.007841,0.401058,0.005974,0.761905,1.899738,0.002829,2.515557
8,(Acupuncture),(destaque),0.007841,0.401058,0.005974,0.761905,1.899738,0.002829,2.515557
622,"(Health & Medical, Massage Therapy)",(destaque),0.009085,0.401058,0.006721,0.739726,1.844437,0.003077,2.301199
162,(Massage Therapy),(destaque),0.009085,0.401058,0.006721,0.739726,1.844437,0.003077,2.301199
925,"(Health & Medical, Beauty & Spas, Massage)",(destaque),0.010454,0.401058,0.007343,0.702381,1.751321,0.003150,2.012446
620,"(Health & Medical, Massage)",(destaque),0.010454,0.401058,0.007343,0.702381,1.751321,0.003150,2.012446
263,"(Shopping, Arts & Entertainment)",(destaque),0.008027,0.401058,0.005352,0.666667,1.662270,0.002132,1.796826
543,"(Event Planning & Services, Shopping)",(destaque),0.009396,0.401058,0.006161,0.655629,1.634749,0.002392,1.739236


As categorias Hotels & Travel, Transportation resultam em um estabelcimento sem destaque com uma confiaça de 90% <br>
Ja categorias como Restaurants, Sports Bars, Dance Clubs, (Food, Ciinese) também resultam em estabelicimentos sem destaque com confiaça acima de 80%.

In [94]:
 #| (dfRegras['consequents'] == {'sem destaque'})
dfRegras= association_rules(frequent_itemsets, metric='confidence', min_threshold=0.75)
dfRegras = dfRegras.sort_values(by='confidence',ascending=False)
aux3 = dfRegras[(dfRegras['consequents'] == {'sem destaque'})]
dfRegras[(dfRegras['consequents'] == {'sem destaque'})]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
184,(Transportation),(sem destaque),0.007218,0.598942,0.006534,0.905172,1.511285,0.002210,4.229338
564,"(Transportation, Hotels & Travel)",(sem destaque),0.007218,0.598942,0.006534,0.905172,1.511285,0.002210,4.229338
440,"(Nightlife, Dance Clubs)",(sem destaque),0.007094,0.598942,0.006161,0.868421,1.449925,0.001912,3.048040
71,(Dance Clubs),(sem destaque),0.007094,0.598942,0.006161,0.868421,1.449925,0.001912,3.048040
61,(Chicken Wings),(sem destaque),0.013068,0.598942,0.011077,0.847619,1.415194,0.003250,2.631942
1009,"(Bars, Nightlife, Restaurants, Sports Bars)",(sem destaque),0.006534,0.598942,0.005538,0.847619,1.415194,0.001625,2.631942
765,"(Bars, Restaurants, Sports Bars)",(sem destaque),0.006534,0.598942,0.005538,0.847619,1.415194,0.001625,2.631942
967,"(Nightlife, Restaurants, Sports Bars)",(sem destaque),0.006534,0.598942,0.005538,0.847619,1.415194,0.001625,2.631942
419,"(Chicken Wings, Restaurants)",(sem destaque),0.013068,0.598942,0.011077,0.847619,1.415194,0.003250,2.631942
629,"(Restaurants, Sports Bars)",(sem destaque),0.006534,0.598942,0.005538,0.847619,1.415194,0.001625,2.631942


In [95]:
#dfRegras[(dfRegras['consequents'] == {'destaque'}) | (dfRegras['consequents'] == {'sem destaque'})].head(30)